In [3]:
!pip install tf-nightly


  Using cached ml_dtypes-0.5.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (21 kB)
Using cached ml_dtypes-0.5.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.7 MB)
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.1 requires ml-dtypes<0.5.0,>=0.3.1, but you have ml-dtypes 0.5.1 which is incompatible.


In [4]:
!pip uninstall ml-dtypes -y
!pip install ml-dtypes==0.4.1



Found existing installation: ml_dtypes 0.5.1
Uninstalling ml_dtypes-0.5.1:
  Successfully uninstalled ml_dtypes-0.5.1
  Using cached ml_dtypes-0.4.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
Using cached ml_dtypes-0.4.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.2 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-nightly 2.20.0.dev20250211 requires ml-dtypes<1.0.0,>=0.5.1, but you have ml-dtypes 0.4.1 which is incompatible.


In [5]:
import tensorflow as tf
import numpy as np
import json
import os


In [6]:
# Create mnist_setup.py file with dataset and model functions
%%writefile mnist_setup.py
import os
import tensorflow as tf
import numpy as np

def mnist_dataset(batch_size):
    (x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
    # Normalize and preprocess the dataset
    x_train = x_train / np.float32(255)
    y_train = y_train.astype(np.int64)
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)) \
        .shuffle(60000).repeat().batch(batch_size)
    return train_dataset

def build_and_compile_cnn_model():
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(28, 28)),
        tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
        tf.keras.layers.Conv2D(32, 3, activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10)
    ])
    model.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
        metrics=['accuracy']
    )
    return model


Writing mnist_setup.py


In [7]:
import mnist_setup

# Set the batch size for a single worker
batch_size = 64
single_worker_dataset = mnist_setup.mnist_dataset(batch_size)
single_worker_model = mnist_setup.build_and_compile_cnn_model()

# Train the model on a single worker
single_worker_model.fit(single_worker_dataset, epochs=3, steps_per_epoch=70)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
AttributeError: module 'ml_dtypes' has no attribute 'float4_e2m1fn'
Epoch 1/3


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


70/70 ━━━━━━━━━━━━━━━━━━━━ 4s 44ms/step - accuracy: 0.1383 - loss: 2.2946
Epoch 2/3
70/70 ━━━━━━━━━━━━━━━━━━━━ 4s 56ms/step - accuracy: 0.3699 - loss: 2.2323
Epoch 3/3
70/70 ━━━━━━━━━━━━━━━━━━━━ 4s 58ms/step - accuracy: 0.5648 - loss: 2.1632


In [8]:
# Multi-Worker Mirrored Strategy setup
strategy = tf.distribute.MultiWorkerMirroredStrategy()

# Configure TF_CONFIG for Multi-Worker Training
tf_config = {
    'cluster': {
        'worker': ['localhost:12345', 'localhost:23456']  # Define two workers on the same machine
    },
    'task': {'type': 'worker', 'index': 0}  # The first worker (index 0)
}

# Set TF_CONFIG environment variable
os.environ['TF_CONFIG'] = json.dumps(tf_config)


In [9]:
# Train the model using MultiWorkerMirroredStrategy
per_worker_batch_size = 64
num_workers = len(tf_config['cluster']['worker'])

global_batch_size = per_worker_batch_size * num_workers
multi_worker_dataset = mnist_setup.mnist_dataset(global_batch_size)

with strategy.scope():
    multi_worker_model = mnist_setup.build_and_compile_cnn_model()

# Train the model across multiple workers
multi_worker_model.fit(multi_worker_dataset, epochs=3, steps_per_epoch=70)


Epoch 1/3
70/70 ━━━━━━━━━━━━━━━━━━━━ 7s 93ms/step - accuracy: 0.1110 - loss: 2.2802
Epoch 2/3
70/70 ━━━━━━━━━━━━━━━━━━━━ 7s 104ms/step - accuracy: 0.3254 - loss: 2.2051
Epoch 3/3
70/70 ━━━━━━━━━━━━━━━━━━━━ 6s 85ms/step - accuracy: 0.5118 - loss: 2.1181


In [10]:
%%writefile main.py
import os
import json
import tensorflow as tf
import mnist_setup

# Set batch size for each worker
per_worker_batch_size = 64
tf_config = json.loads(os.environ['TF_CONFIG'])
num_workers = len(tf_config['cluster']['worker'])

# Initialize MultiWorkerMirroredStrategy
strategy = tf.distribute.MultiWorkerMirroredStrategy()

global_batch_size = per_worker_batch_size * num_workers
multi_worker_dataset = mnist_setup.mnist_dataset(global_batch_size)

with strategy.scope():
    multi_worker_model = mnist_setup.build_and_compile_cnn_model()

multi_worker_model.fit(multi_worker_dataset, epochs=3, steps_per_epoch=70)


Writing main.py


In [11]:
os.environ['TF_CONFIG'] = json.dumps(tf_config)
!python main.py &> job_0.log &


In [12]:
# Change the worker index to 1 for the second worker
tf_config['task']['index'] = 1
os.environ['TF_CONFIG'] = json.dumps(tf_config)
!python main.py


2025-02-18 10:38:34.952448: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
I0000 00:00:1739875114.962322    2510 grpc_server_lib.cc:463] Started server with target: grpc://localhost:23456
I0000 00:00:1739875114.965457    2510 coordination_service_agent.cc:369] Coordination agent has successfully connected.
AttributeError: module 'ml_dtypes' has no attribute 'float4_e2m1fn'
2025-02-18 10:38:36.329372: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
2025-02-18 10:38:36.930076: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.
2025-02-18 10:38:37.174309: W external/loc

In [13]:
# Change the worker index to 1 for the second worker
tf_config['task']['index'] = 1
os.environ['TF_CONFIG'] = json.dumps(tf_config)
!python main.py


2025-02-18 10:38:59.747699: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
I0000 00:00:1739875139.756805    2763 grpc_server_lib.cc:463] Started server with target: grpc://localhost:23456
2025-02-18 11:39:04.759278: E tensorflow/core/common_runtime/base_collective_executor.cc:248] BaseCollectiveExecutor::StartAbort DEADLINE_EXCEEDED: Deadline Exceeded

RPC: /tensorflow.CoordinationService/RegisterTask
E0000 00:00:1739878744.759543    2763 context_distributed_manager.cc:1107] Deadline Exceeded
Additional GRPC error information from remote target /job:worker/replica:0/task:0 while calling /tensorflow.CoordinationService/RegisterTask:
:{"created":"@1739878744.756606172","description":"Deadline Exceeded","file":"external/com_github_grpc_grpc/src/core/ext/filters/deadline/deadline_filter.cc","file_line":69,"grpc_status":4}
Traceback (most recent call last):
  File "/content/main.py", 

In [14]:
callbacks = [tf.keras.callbacks.BackupAndRestore(backup_dir='/tmp/backup')]

with strategy.scope():
    multi_worker_model = mnist_setup.build_and_compile_cnn_model()

multi_worker_model.fit(multi_worker_dataset, epochs=3, steps_per_epoch=70, callbacks=callbacks)


Epoch 1/3
70/70 ━━━━━━━━━━━━━━━━━━━━ 8s 112ms/step - accuracy: 0.1120 - loss: 2.2961
Epoch 2/3
70/70 ━━━━━━━━━━━━━━━━━━━━ 8s 113ms/step - accuracy: 0.3346 - loss: 2.2340
Epoch 3/3
70/70 ━━━━━━━━━━━━━━━━━━━━ 8s 108ms/step - accuracy: 0.5356 - loss: 2.1709


In [ ]:
!cat job_0.log


In [ ]:
model_path = '/tmp/keras_model'

def _is_chief(task_type, task_id):
    return task_type == 'worker' and task_id == 0

def write_filepath(filepath, task_type, task_id):
    if not _is_chief(task_type, task_id):
        temp_dir = f"workertemp_{task_id}"
        tf.io.gfile.makedirs(temp_dir)
        return os.path.join(temp_dir, os.path.basename(filepath))
    return filepath

task_type, task_id = strategy.cluster_resolver.task_type, strategy.cluster_resolver.task_id
write_model_path = write_filepath(model_path, task_type, task_id)
multi_worker_model.save(write_model_path)


In [15]:
os.environ.pop('TF_CONFIG', None)



'{"cluster": {"worker": ["localhost:12345", "localhost:23456"]}, "task": {"type": "worker", "index": 1}}'